In [0]:
orders_raw_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/AggregateFunctions/Order_Raw_csv.csv")

In [0]:
orders_raw_df.show()

+------+--------------+----------+----------+--------------+-----------+---------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Product ID|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+---------------+--------+--------+--------+--------+
|     1|CA-2016-152156|08-11-2016|11-11-2016|  Second Class|   CG-12520|FUR-BO-10001798|  261.96|       2|       0| 41.9136|
|     2|CA-2016-152156|08-11-2016|11-11-2016|  Second Class|   CG-12520|FUR-CH-10000454|  731.94|       3|       0| 219.582|
|     3|CA-2016-138688|12-06-2016|16-06-2016|  Second Class|   DV-13045|OFF-LA-10000240|   14.62|       2|       0|  6.8714|
|     4|US-2015-108966|11-10-2015|18-10-2015|Standard Class|   SO-20335|FUR-TA-10000577|957.5775|       5|    0.45|-383.031|
|     5|US-2015-108966|11-10-2015|18-10-2015|Standard Class|   SO-20335|OFF-ST-10000760|  22.368|       2|     0.2|  2.5164|


In [0]:
from pyspark.sql.functions import *

In [0]:
orders_cols_renamed = orders_raw_df.select(col("Row ID").alias("Row_Id"),col("Order ID").alias("Order_Id"),col("Order Date").alias("Order_Date"),col("Ship Date").alias("Ship_Date"),col("Ship Mode").alias("Ship_Mode"),col("Customer ID").alias("Customer_Id"),col("Product ID").alias("Product_Id"),col("Sales"),col("Quantity"),col("Discount"),col("Profit"))

In [0]:
orders_cols_renamed.show()

+------+--------------+----------+----------+--------------+-----------+---------------+--------+--------+--------+--------+
|Row_Id|      Order_Id|Order_Date| Ship_Date|     Ship_Mode|Customer_Id|     Product_Id|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+---------------+--------+--------+--------+--------+
|     1|CA-2016-152156|08-11-2016|11-11-2016|  Second Class|   CG-12520|FUR-BO-10001798|  261.96|       2|       0| 41.9136|
|     2|CA-2016-152156|08-11-2016|11-11-2016|  Second Class|   CG-12520|FUR-CH-10000454|  731.94|       3|       0| 219.582|
|     3|CA-2016-138688|12-06-2016|16-06-2016|  Second Class|   DV-13045|OFF-LA-10000240|   14.62|       2|       0|  6.8714|
|     4|US-2015-108966|11-10-2015|18-10-2015|Standard Class|   SO-20335|FUR-TA-10000577|957.5775|       5|    0.45|-383.031|
|     5|US-2015-108966|11-10-2015|18-10-2015|Standard Class|   SO-20335|OFF-ST-10000760|  22.368|       2|     0.2|  2.5164|


In [0]:
orders_cols_renamed.createOrReplaceTempView("orders_raw_tbl")

In [0]:
spark.sql("Select * from orders_raw_tbl").show()

+------+--------------+----------+----------+--------------+-----------+---------------+--------+--------+--------+--------+
|Row_Id|      Order_Id|Order_Date| Ship_Date|     Ship_Mode|Customer_Id|     Product_Id|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+---------------+--------+--------+--------+--------+
|     1|CA-2016-152156|08-11-2016|11-11-2016|  Second Class|   CG-12520|FUR-BO-10001798|  261.96|       2|       0| 41.9136|
|     2|CA-2016-152156|08-11-2016|11-11-2016|  Second Class|   CG-12520|FUR-CH-10000454|  731.94|       3|       0| 219.582|
|     3|CA-2016-138688|12-06-2016|16-06-2016|  Second Class|   DV-13045|OFF-LA-10000240|   14.62|       2|       0|  6.8714|
|     4|US-2015-108966|11-10-2015|18-10-2015|Standard Class|   SO-20335|FUR-TA-10000577|957.5775|       5|    0.45|-383.031|
|     5|US-2015-108966|11-10-2015|18-10-2015|Standard Class|   SO-20335|OFF-ST-10000760|  22.368|       2|     0.2|  2.5164|


Simple Aggregations -
Count Total Number of Records (Programmatic way)

In [0]:
orders_cols_renamed.select(count("*").alias("Row Count")).show()

+---------+
|Row Count|
+---------+
|     9994|
+---------+



In [0]:
orders_cols_renamed.selectExpr("count(*) as row_count").show()

+---------+
|row_count|
+---------+
|     9994|
+---------+



Simple Aggregation - Count Total Number of Records (SQL style)

In [0]:
spark.sql("select count(*) as row_count from orders_raw_tbl").show()

+---------+
|row_count|
+---------+
|     9994|
+---------+



Simple Aggreation - Count Distinct Orders(Programmatic style)

In [0]:
orders_cols_renamed.select(countDistinct("Order_Id").alias("Unique orders")).show()

+-------------+
|Unique orders|
+-------------+
|         5009|
+-------------+



In [0]:
orders_cols_renamed.selectExpr("count(distinct(Order_Id)) as Unique_orders").show()

+-------------+
|Unique_orders|
+-------------+
|         5009|
+-------------+



Simple Aggreation - Count Distinct Orders(SQL style)

In [0]:
spark.sql("select count(distinct(Order_id)) as unique_orders from orders_raw_tbl").show()

+-------------+
|unique_orders|
+-------------+
|         5009|
+-------------+



Grouping Aggregation - Sum of Profict per customer, per product sorted by customer id (Programmatic style)

In [0]:
orders_cols_renamed.groupBy("Customer_id","Product_Id").agg(sum("Profit")).sort("customer_Id").show()

+-----------+---------------+-----------+
|Customer_id|     Product_Id|sum(Profit)|
+-----------+---------------+-----------+
|   AA-10315|OFF-ST-10001590|     7.0096|
|   AA-10315|OFF-SU-10000151|  -786.0144|
|   AA-10315|OFF-AP-10000576|     90.735|
|   AA-10315|OFF-AP-10002765|    14.8344|
|   AA-10315|TEC-PH-10000895|    32.3982|
|   AA-10315|OFF-BI-10004654|       5.77|
|   AA-10315|OFF-PA-10000349|     7.0218|
|   AA-10315|FUR-FU-10002456|     6.2608|
|   AA-10315|OFF-FA-10001332|     0.7776|
|   AA-10315|OFF-BI-10004390|    252.588|
|   AA-10315|TEC-AC-10002857|     5.7365|
|   AA-10375|OFF-PA-10002479|      2.376|
|   AA-10375|OFF-BI-10002498|     5.0463|
|   AA-10375|OFF-BI-10002706|    11.1384|
|   AA-10375|TEC-AC-10004145|   114.9954|
|   AA-10375|OFF-AR-10000127|     0.5904|
|   AA-10375|TEC-PH-10004522|     29.245|
|   AA-10375|OFF-PA-10002659|      7.774|
|   AA-10375|OFF-ST-10001325|     1.4672|
|   AA-10375|OFF-PA-10000157|    18.7812|
+-----------+---------------+-----

Grouping Aggregation - Sum of Profict per customer, per product sorted by customer id (SQL style)

In [0]:
spark.sql("select Customer_id, Product_Id, sum(Profit) as Total_Profict from orders_raw_tbl group by Customer_Id, Product_Id order by Customer_Id").show()

+-----------+---------------+-------------+
|Customer_id|     Product_Id|Total_Profict|
+-----------+---------------+-------------+
|   AA-10315|OFF-ST-10001590|       7.0096|
|   AA-10315|OFF-SU-10000151|    -786.0144|
|   AA-10315|OFF-AP-10000576|       90.735|
|   AA-10315|OFF-AP-10002765|      14.8344|
|   AA-10315|TEC-PH-10000895|      32.3982|
|   AA-10315|OFF-BI-10004654|         5.77|
|   AA-10315|OFF-PA-10000349|       7.0218|
|   AA-10315|FUR-FU-10002456|       6.2608|
|   AA-10315|OFF-FA-10001332|       0.7776|
|   AA-10315|OFF-BI-10004390|      252.588|
|   AA-10315|TEC-AC-10002857|       5.7365|
|   AA-10375|OFF-PA-10002479|        2.376|
|   AA-10375|OFF-BI-10002498|       5.0463|
|   AA-10375|OFF-BI-10002706|      11.1384|
|   AA-10375|TEC-AC-10004145|     114.9954|
|   AA-10375|OFF-AR-10000127|       0.5904|
|   AA-10375|TEC-PH-10004522|       29.245|
|   AA-10375|OFF-PA-10002659|        7.774|
|   AA-10375|OFF-ST-10001325|       1.4672|
|   AA-10375|OFF-PA-10000157|   